In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Concatenate, Dense, Dropout, Lambda, BatchNormalization, GlobalAveragePooling2D, concatenate
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import pickle
import gzip
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.utils import class_weight
import matplotlib.pyplot as plt


# Load in dataframe
This df contains atomic features, soap descriptors, and bandgap

In [ ]:
data_path = "/Users/cadenmyers/billingelab/dev/ml4ms_bandgap_final/data/soap_and_atomic_features.pkl.gz"
# Step 1: Read the compressed pickle
with gzip.open(data_path, 'rb') as f:
    data_df = pickle.load(f)

print(data_df.shape)
# data_df = data_df[data_df['gap opt'] >= 0.2]
data_df = data_df.dropna()
print(data_df.shape)
data_df.head()

# Visualize data

In [ ]:
import seaborn as sns

# Define the features to plot
features = [
    'electronegativity_mean', 'electronegativity_max', 'electronegativity_min', 'electronegativity_std',
    'atomic_radius_mean', 'atomic_radius_max', 'atomic_radius_min', 'atomic_radius_std',
    'ionenergies_mean', 'ionenergies_max', 'ionenergies_min', 'ionenergies_std',
    'covalent_radius_mean', 'covalent_radius_max', 'covalent_radius_min', 'covalent_radius_std',
    'nvalence_mean', 'nvalence_max', 'nvalence_min', 'nvalence_std'
]

# Set up the grid
fig, axes = plt.subplots(4, 5, figsize=(20, 12))
axes = axes.flatten()

# Plot each feature
for i, feature in enumerate(features):
    sns.histplot(data_df[feature], ax=axes[i], kde=True, bins=100, color='steelblue')
    axes[i].set_title(feature, fontsize=10)
    axes[i].tick_params(labelsize=8)

plt.tight_layout()
plt.suptitle("Feature Distributions", fontsize=16, y=1.02)
plt.show()

In [ ]:
print(data_df.columns)

In [ ]:
has_gap = data_df['gap opt'] > 1
plt.hist(has_gap.astype(int), bins=2, edgecolor='black')
plt.xticks([0, 1], ['No Gap', 'Has Gap'])

In [ ]:
X_atomic_8 = data_df[[
    'electronegativity_mean', 'electronegativity_std', 
    'atomic_radius_mean', 'atomic_radius_std',
    'ionenergies_mean', 'ionenergies_std', 
    'covalent_radius_mean', 'covalent_radius_std',
]]
soaps = np.array(data_df['padded_soap'].tolist())
X_soap_2d = soaps[..., np.newaxis]  # add channel dim: (N, 64, 800, 1)
X_atomic_8 = X_atomic_8.to_numpy()
print('X_soap_2d shape:', X_soap_2d.shape)
print('X_atomic_8 shape:', X_atomic_8.shape)
print('--------------------')

# Step 2: Split the data into training and testing sets
X_soap_train, X_soap_test, X_atomic_8_train, X_atomic_8_test, y_train, y_test = train_test_split(
    X_soap_2d, X_atomic_8, data_df['gap opt'], test_size=0.2, random_state=42
)

bg_threshold = 0.02 # eV
y_train_binary = (y_train > bg_threshold).astype(int)
y_test_binary = (y_test > bg_threshold).astype(int)

# Flatten the soap descriptors for scaling
X_soap_train_flat = X_soap_train.reshape(X_soap_train.shape[0], -1)
X_soap_test_flat = X_soap_test.reshape(X_soap_test.shape[0], -1)

# scale soap descriptors
scaler_soap = MinMaxScaler()

# Fit and transform the training set, and transform the test set
X_soap_train_scaled = scaler_soap.fit_transform(X_soap_train_flat)
X_soap_test_scaled = scaler_soap.transform(X_soap_test_flat)

# Reshape back to the original shape (N, 64, 800, 1)
X_soap_train_scaled = X_soap_train_scaled.reshape(X_soap_train.shape)
X_soap_test_scaled = X_soap_test_scaled.reshape(X_soap_test.shape)

# scale atomic input data
scaler_atomic_8 = MinMaxScaler()

X_atomic_8_train_scaled = scaler_atomic_8.fit_transform(X_atomic_8_train)
X_atomic_8_test_scaled = scaler_atomic_8.transform(X_atomic_8_test)

print('X_soap_train_scaled shape:', X_soap_train_scaled.shape)
print('X_soap_test_scaled shape:', X_soap_test_scaled.shape)
print('X_atomic_8_train_scaled shape:', X_atomic_8_train_scaled.shape)
print('X_atomic_8_test_scaled shape:', X_atomic_8_test_scaled.shape)
print('--------------------')
print('y_train_binary shape:', y_train_binary.shape)
print('y_test_binary shape:', y_test_binary.shape)

In [ ]:
X_atomic_20 = data_df.drop(columns=['formula', 'mpid', 'gap opt', 'padded_soap', 'soap_flat']).to_numpy()
print('X_soap_2d shape:', X_soap_2d.shape)
print('X_atomic_20 shape:', X_atomic_20.shape)
print('--------------------')
# Step 2: Split the data into training and testing sets
_, _, X_atomic_20_train, X_atomic_20_test, y_train, y_test = train_test_split(
    X_soap_2d, X_atomic_20, data_df['gap opt'], test_size=0.2, random_state=42
)

# scale atomic input data
scaler_atomic_20 = MinMaxScaler()

X_atomic_20_train_scaled = scaler_atomic_20.fit_transform(X_atomic_20_train)
X_atomic_20_test_scaled = scaler_atomic_20.transform(X_atomic_20_test)

print('X_soap_train_scaled shape:', X_soap_train_scaled.shape)
print('X_soap_test_scaled shape:', X_soap_test_scaled.shape)
print('---------------------')
print('X_atomic_20_train_scaled shape:', X_atomic_20_train_scaled.shape)
print('X_atomic_20_test_scaled shape:', X_atomic_20_test_scaled.shape)
print('X_atomic_8_train_scaled shape:', X_atomic_8_train_scaled.shape)
print('X_atomic_8_test_scaled shape:', X_atomic_8_test_scaled.shape)
print('--------------------')
print('y_train_binary shape:', y_train_binary.shape)
print('y_test_binary shape:', y_test_binary.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

In [ ]:
# SOAP input branch
soap_input = Input(shape=(64, 800, 1), name='soap_input')
x = Conv2D(32, (3, 3), activation='relu')(soap_input)
x = MaxPooling2D((2, 2))(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = BatchNormalization()(x)
x = GlobalAveragePooling2D()(x)

# Periodic table input (8 features)
pt_input = Input(shape=(8,), name='periodic_features')

# Feature group slices (each group has 2 features)
electronegativity = Lambda(lambda t: t[:, 0:2])(pt_input)
atomic_radius     = Lambda(lambda t: t[:, 2:4])(pt_input)
ion_energies      = Lambda(lambda t: t[:, 4:6])(pt_input)
covalent_radius   = Lambda(lambda t: t[:, 6:8])(pt_input)

# Process each group with a small Dense layer
e_dense = Dense(8, activation='relu')(electronegativity)
r_dense = Dense(8, activation='relu')(atomic_radius)
i_dense = Dense(8, activation='relu')(ion_energies)
c_dense = Dense(8, activation='relu')(covalent_radius)

# Concatenate all group representations
y = Concatenate()([e_dense, r_dense, i_dense, c_dense])
y = Dense(64, activation='relu')(y)
y = BatchNormalization()(y)
y = Dense(32, activation='relu')(y)

# Merge with SOAP branch
combined = Concatenate()([x, y])
z = Dense(128, activation='relu')(combined)
z = Dropout(0.3)(z)
z = Dense(1, activation='sigmoid')(z)

# Final model
model = Model(inputs=[soap_input, pt_input], outputs=z)
model.compile(optimizer=Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy', Precision(), Recall(), AUC()])

model.summary()

# Compute class weights from labels
y_train_array = np.array(y_train_binary)

# Correctly format the `classes` parameter
classes = np.unique(y_train_array)

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=y_train_array
)
class_weights_dict = {0: class_weights[0], 1: class_weights[1]}
print(class_weights_dict)

In [ ]:
callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        min_delta=0.001,  # Minimum accuracy improvement required
        patience=5,       # Number of epochs with no improvement to wait
        verbose=1,        # Display messages when stopping
        restore_best_weights=True
    )
]

history = model.fit(
    {'soap_input': X_soap_train, 'periodic_features': X_atomic_train},
    y_train_binary,
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    class_weight=class_weights_dict,
)


In [ ]:
model.save('/Users/cadenmyers/billingelab/dev/ml4ms_bandgap_final/models/bandgap_classifier_pt_slicing_std_avg.h5')
model.save('/Users/cadenmyers/billingelab/dev/ml4ms_bandgap_final/models/bandgap_classifier_pt_slicing_std_avg.keras')

In [ ]:
test_loss, test_acc, test_precision, test_recall, test_auc = model.evaluate(
    {'soap_input': X_soap_test, 'periodic_features': X_atomic_test},
    y_test_binary,  # Use binary labels
    verbose=0
)
print(f"Test Accuracy: {test_acc:.3f}, Precision: {test_precision:.3f}, Recall: {test_recall:.3f}, AUC: {test_auc:.3f}")

In [ ]:
results = model.evaluate(
    {'soap_input': X_soap_test, 'periodic_features': X_atomic_test},
    y_test_binary,
    verbose=2
)

print("\nEvaluation metrics:", dict(zip(model.metrics_names, results)))

# prediction
y_pred_probs = model.predict({'soap_input': X_soap_test, 'periodic_features': X_atomic_test})
y_pred = (y_pred_probs > 0.5).astype(int)

print("\nClassification Report:")
print(classification_report(y_test_binary, y_pred, target_names=['No Gap', 'Has Gap']))

print("\nConfusion Matrix:")
plt.imshow(confusion_matrix(y_test_binary, y_pred), cmap='RdBu', interpolation='nearest')
plt.colorbar()
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# print(confusion_matrix(y_test_binary, y_pred))

# Best performing model based on f1

In [ ]:
# Inputs
soap_input = Input(shape=(64, 800, 1), name='soap_input')
periodic_features = Input(shape=(8,), name='periodic_features')

# ───────────────────────────────────────────────
# CNN on SOAP input
x = Conv2D(32, kernel_size=(3, 3), activation='relu')(soap_input)              # (62, 798, 32)
x = MaxPooling2D(pool_size=(2, 2))(x)                                          # (31, 399, 32)
x = BatchNormalization()(x)

x = Conv2D(64, kernel_size=(3, 3), activation='relu')(x)                       # (29, 397, 64)
x = MaxPooling2D(pool_size=(2, 2))(x)                                          # (14, 198, 64)
x = BatchNormalization()(x)

x = GlobalAveragePooling2D()(x)                                               # (64,)

# ───────────────────────────────────────────────
# Dense on periodic features
y = Dense(64, activation='relu')(periodic_features)
y = BatchNormalization()(y)                                                    # (64,)

# ───────────────────────────────────────────────
# Combine
combined = Concatenate()([x, y])                                               # (128,)
combined = Dense(128, activation='relu')(combined)
combined = Dropout(0.5)(combined)  # adjust if needed
output = Dense(1, activation='sigmoid')(combined)

# ───────────────────────────────────────────────
# Build model
model = Model(inputs=[soap_input, periodic_features], outputs=output)

# ───────────────────────────────────────────────
# (Optional) Compile
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# ───────────────────────────────────────────────
# Summary
model.summary()

In [204]:
callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        min_delta=0.001,  # Minimum accuracy improvement required
        patience=5,       # Number of epochs with no improvement to wait
        verbose=1,        # Display messages when stopping
        restore_best_weights=True
    )
]

history = model.fit(
    {'soap_input': X_soap_train, 'periodic_features': X_atomic_8_train},
    y_train_binary,
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    class_weight=class_weights_dict,
    callbacks=callbacks,
)

Epoch 1/30
107/107 ━━━━━━━━━━━━━━━━━━━━ 114s 1s/step - accuracy: 0.7632 - loss: 0.5855 - val_accuracy: 0.8285 - val_loss: 0.4291
Epoch 2/30
107/107 ━━━━━━━━━━━━━━━━━━━━ 114s 1s/step - accuracy: 0.7972 - loss: 0.4695 - val_accuracy: 0.8448 - val_loss: 0.3748
Epoch 3/30
107/107 ━━━━━━━━━━━━━━━━━━━━ 113s 1s/step - accuracy: 0.8063 - loss: 0.4460 - val_accuracy: 0.8495 - val_loss: 0.3804
Epoch 4/30
107/107 ━━━━━━━━━━━━━━━━━━━━ 111s 1s/step - accuracy: 0.8262 - loss: 0.4354 - val_accuracy: 0.8156 - val_loss: 0.4441
Epoch 5/30
107/107 ━━━━━━━━━━━━━━━━━━━━ 111s 1s/step - accuracy: 0.8234 - loss: 0.4433 - val_accuracy: 0.7468 - val_loss: 0.5335
Epoch 6/30
107/107 ━━━━━━━━━━━━━━━━━━━━ 108s 1s/step - accuracy: 0.8218 - loss: 0.4393 - val_accuracy: 0.8541 - val_loss: 0.3475
Epoch 7/30
107/107 ━━━━━━━━━━━━━━━━━━━━ 107s 1s/step - accuracy: 0.8245 - loss: 0.4108 - val_accuracy: 0.8483 - val_loss: 0.3460
Epoch 8/30
107/107 ━━━━━━━━━━━━━━━━━━━━ 108s 1s/step - accuracy: 0.8321 - loss: 0.4050 - val_accu

In [205]:
results = model.evaluate(
    {'soap_input': X_soap_test, 'periodic_features': X_atomic_8_test},
    y_test_binary,
    verbose=2
)

print("\nEvaluation metrics:", dict(zip(model.metrics_names, results)))

# prediction
y_pred_probs = model.predict({'soap_input': X_soap_test, 'periodic_features': X_atomic_8_test})
y_pred = (y_pred_probs > 0.5).astype(int)

print("\nClassification Report:")
print(classification_report(y_test_binary, y_pred, target_names=['No Gap', 'Has Gap']))


34/34 - 8s - 235ms/step - accuracy: 0.8422 - loss: 0.3711

Evaluation metrics: {'loss': 0.3710688650608063, 'compile_metrics': 0.8422035574913025}
34/34 ━━━━━━━━━━━━━━━━━━━━ 7s 218ms/step

Classification Report:
              precision    recall  f1-score   support

      No Gap       0.71      0.74      0.73       301
     Has Gap       0.90      0.88      0.89       770

    accuracy                           0.84      1071
   macro avg       0.80      0.81      0.81      1071
weighted avg       0.84      0.84      0.84      1071



In [206]:
model.save('/Users/cadenmyers/billingelab/dev/ml4ms_bandgap_final/models/bandgap_classifier_pt_no_slicing_std_avg.h5')

# Neural Net only using atomic info

In [ ]:
# Define periodic table feature columns
pt_columns = [
    'electronegativity_mean', 'electronegativity_max', 'electronegativity_min', 'electronegativity_std',
    'atomic_radius_mean', 'atomic_radius_max', 'atomic_radius_min', 'atomic_radius_std',
    'ionenergies_mean', 'ionenergies_max', 'ionenergies_min', 'ionenergies_std',
    'covalent_radius_mean', 'covalent_radius_max', 'covalent_radius_min', 'covalent_radius_std',
    'nvalence_mean', 'nvalence_max', 'nvalence_min', 'nvalence_std'
]

# Extract features and labels
X = data_df[pt_columns].values
y = data_df['gap opt'].values

print('y shape:', y.shape)
print('X shape:', X.shape)
print('----------------')
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


print('X_train_scaled shape:', X_train_scaled.shape)
print('X_test_scaled shape:', X_test_scaled.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

# Compute class weights
classes = np.unique(y_train)
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=y_train
)
class_weights_dict = dict(zip(classes, class_weights))


In [ ]:

# Build NN model
pt_input = Input(shape=(X_train_scaled.shape[1],), name='periodic_features')
x = Dense(64, activation='relu')(pt_input)
x = BatchNormalization()(x)
x = Dense(64, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=pt_input, outputs=output)
model.compile(optimizer=Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy', Precision(), Recall(), AUC()])

# Model summary
model.summary()





In [ ]:
# Train
model.fit(X_train_scaled, y_train,
          validation_data=(X_test_scaled, y_test),
          epochs=50,
          batch_size=32,
          class_weight=class_weights_dict,
          verbose=2)


In [ ]:
# Evaluate
results = model.evaluate(X_test_scaled, y_test, verbose=0)
print(dict(zip(model.metrics_names, results)))

In [ ]:
# plt.plot(X_combined[0])

In [ ]:
xgb = XGBRegressor(verbosity=2)
xgb.fit(X_train, y_train)

In [ ]:
# model_path = "/Users/cadenmyers/billingelab/dev/ml4ms_bandgap_final/data/xgb_model_all_data_oob.pkl"
# with open(model_path, "wb") as model_file:
#     pickle.dump(xgb, model_file)
# print(f"Model saved to {model_path}")

In [ ]:
xgb_pred = xgb.predict(X_test)
xgb_mae = mean_absolute_error(y_test, xgb_pred)
print(f"XGBoost MAE: {xgb_mae:.4f}")
# Plotting the predictions
plt.figure(figsize=(5, 5))
plt.scatter(y_test, xgb_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('True bandgap')
plt.ylabel('predicted bandgap')


In [ ]:
y_pred = model.predict({'soap_input': X_soap_test, 'periodic_features': X_atomic_test})

# Parity plot for predicted vs true bandgap values (Neural Network)
plt.figure(figsize=(6, 6))
plt.scatter(y_test, y_pred, alpha=0.5, label='Predictions')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='Ideal')
plt.xlabel('True Bandgap (eV)')
plt.ylabel('Predicted Bandgap (eV)')
plt.title('Parity Plot (Neural Network)')
plt.legend()
plt.grid(True)
plt.show()